# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
# Read tic-tac-toe.csv into a dataframe.
import pandas as pd

tictactoe = pd.read_csv('tic-tac-toe.csv')
tictactoe.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [2]:
# Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
tictactoe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB


In [3]:
# Convert the categorical values to numeric in all columns.
# Dummies are already normalized data

tictactoe_df = pd.get_dummies(tictactoe)
tictactoe_df

,class,TL_b,TL_o,TL_x,TM_b,TM_o,TM_x,TR_b,TR_o,TR_x,...,MR_x,BL_b,BL_o,BL_x,BM_b,BM_o,BM_x,BR_b,BR_o,BR_x
0,True,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,1,0,0,1,0
1,True,0,0,1,0,0,1,0,0,1,...,0,0,1,0,0,0,1,0,1,0
2,True,0,0,1,0,0,1,0,0,1,...,0,0,1,0,0,1,0,0,0,1
3,True,0,0,1,0,0,1,0,0,1,...,0,0,1,0,1,0,0,1,0,0
4,True,0,0,1,0,0,1,0,0,1,...,0,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,False,0,1,0,0,0,1,0,0,1,...,0,0,1,0,0,0,1,0,0,1
954,False,0,1,0,0,0,1,0,1,0,...,0,0,0,1,0,1,0,0,0,1
955,False,0,1,0,0,0,1,0,1,0,...,1,0,0,1,0,1,0,0,0,1
956,False,0,1,0,0,0,1,0,1,0,...,1,0,0,1,0,1,0,0,0,1


In [4]:
# Separate the inputs and output.
data = tictactoe_df.drop(columns='class')
target = tictactoe_df['class']

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [42]:
# Split the training and test data.
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.2, random_state=0)

In [71]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

# Create a Sequential model.
model = keras.Sequential

# Add layers
model = Sequential([
                    Dense(2000, activation="relu",name="layer1"), 
                    Dense(15, activation="relu",name="layer2"),
                    Dense(15, activation="relu",name="layer3")])

model.add(Dense(3, activation='softmax',name="output"))

In [72]:
# Compile the model
model.compile(
              optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
             )

In [73]:
# Fit the training data.

history = model.fit(
  X_train,
  y_train,
  epochs=50,
  batch_size=50,
)

Epoch 1/50
16/16 [==============================] - 1s 3ms/step - loss: 0.9690 - accuracy: 0.6749
Epoch 2/50
16/16 [==============================] - 0s 4ms/step - loss: 0.6685 - accuracy: 0.7702
Epoch 3/50
16/16 [==============================] - 0s 4ms/step - loss: 0.4322 - accuracy: 0.8211
Epoch 4/50
16/16 [==============================] - 0s 4ms/step - loss: 0.2838 - accuracy: 0.9308
Epoch 5/50
16/16 [==============================] - 0s 4ms/step - loss: 0.2056 - accuracy: 0.9556
Epoch 6/50
16/16 [==============================] - 0s 4ms/step - loss: 0.1414 - accuracy: 0.9791
Epoch 7/50
16/16 [==============================] - 0s 4ms/step - loss: 0.1075 - accuracy: 0.9830
Epoch 8/50
16/16 [==============================] - 0s 4ms/step - loss: 0.0676 - accuracy: 0.9909
Epoch 9/50
16/16 [==============================] - 0s 4ms/step - loss: 0.0466 - accuracy: 0.9935
Epoch 10/50
16/16 [==============================] - 0s 4ms/step - loss: 0.0391 - accuracy: 0.9922
Epoch 11/50
16/16 [

In [74]:
# Evaluate your neural network model with the test data.

model.evaluate(X_test,y_test)

6/6 [==============================] - 0s 2ms/step - loss: 0.0351 - accuracy: 0.9844


[0.0350521095097065, 0.984375]

In [75]:
# Save your model as tic-tac-toe.model.

model.save_weights('tic-tac-toe.model')

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [76]:
# Loading model
model.load_weights('tic-tac-toe.model')

# making predictions
predictions = model.predict(X_test[:10])

import numpy as np
print(np.argmax(predictions, axis=1)) 

[0 1 1 1 1 0 1 1 0 1]


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [66]:
# Filipe's solution came to an 100% accuracy:

model = keras.Sequential()

model.add(Dense(2000, input_dim=27, activation='relu'))
model.add(Dense(2, activation='relu'))
model.add(Dense(2, activation='relu'))
model.add(Dense(2, activation=tf.nn.softmax))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics='accuracy')

model.fit(X_train, y_train, epochs=20)

val_loss, val_acc = model.evaluate(X_test, y_test)

pred = model.predict(X_test[:10])
print(np.argmax(pred, axis=1)) 

Epoch 1/20
24/24 [==============================] - 1s 4ms/step - loss: 0.6899 - accuracy: 0.6475
Epoch 2/20
24/24 [==============================] - 0s 3ms/step - loss: 0.6834 - accuracy: 0.6632
Epoch 3/20
24/24 [==============================] - 0s 4ms/step - loss: 0.6775 - accuracy: 0.6632
Epoch 4/20
24/24 [==============================] - 0s 4ms/step - loss: 0.6723 - accuracy: 0.6632
Epoch 5/20
24/24 [==============================] - 0s 3ms/step - loss: 0.6679 - accuracy: 0.6632
Epoch 6/20
24/24 [==============================] - 0s 3ms/step - loss: 0.6637 - accuracy: 0.6632
Epoch 7/20
24/24 [==============================] - 0s 4ms/step - loss: 0.6603 - accuracy: 0.6632
Epoch 8/20
24/24 [==============================] - 0s 3ms/step - loss: 0.6571 - accuracy: 0.6632
Epoch 9/20
24/24 [==============================] - 0s 4ms/step - loss: 0.6545 - accuracy: 0.6632
Epoch 10/20
24/24 [==============================] - 0s 3ms/step - loss: 0.6522 - accuracy: 0.6632
Epoch 11/20
24/24 [

**Which approach(es) did you find helpful to improve your model performance?**

Comparing the two differente approaches, I saw that our TA used differente parameters, such as input_dim. There is a great repositopry for an example of Keras usage:

https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-find-the-value-for-keras-input_shape-input_dim.md